# Temperature forecast data example (Turku Artukainen)

Info about FMI open data https://en.ilmatieteenlaitos.fi/open-data

In [1]:
import datetime
import pandas as pd
import re
import requests
import xml.etree.ElementTree as et
from io import StringIO

api_url = 'https://opendata.fmi.fi/wfs'

In [2]:
def get_fmidata(parameters):
    r = requests.get(f'{api_url}?{parameters}')
    # XML root and namespaces
    root = et.fromstring(r.text)
    namespaces = dict([node for _, node in et.iterparse(StringIO(r.text), events=['start-ns'])])
    # Extract name list
    names = list(map(lambda f: f.attrib['name'] ,root.findall('.//swe:field', namespaces)))
    # Extract Unix timestamps
    timestamps = re.split(r'\s+',root.find('.//gmlcov:positions', namespaces).text)[3:-1:3]
    # Convert Unix timestamps to datetimes with Helsinki timezone
    datetimeindex = pd.to_datetime(sorted(timestamps*len(names)), unit='s')
    datetimeindex = datetimeindex.tz_localize(tz='UTC').tz_convert('Europe/Helsinki')
    # Extract data
    data = re.split(r'\s+',root.find('.//gml:doubleOrNilReasonTupleList', namespaces).text)[1:-1]
    # Get URL for and print property explanations
    property_url = root.find('.//om:observedProperty', namespaces).attrib[
        '{http://www.w3.org/1999/xlink}href']
    print(f'Properties: {property_url}')
    # Create and return DataFrame
    df = pd.DataFrame({
        'name': names*len(timestamps), 
        'data': data}, 
        index=datetimeindex)
    return df

In [3]:
# Get geoids from https://www.geonames.org
geoid = 660972 # Turku, Artukainen
# List of stored queries https://ilmatieteenlaitos.fi/tallennetut-kyselyt
query = 'fmi::forecast::hirlam::surface::point::multipointcoverage'
df = get_fmidata(f'request=getFeature&storedquery_id={query}&geoid={geoid}')
temperature = df[df['name']=='Temperature']
temperature

Properties: https://opendata.fmi.fi/meta?observableProperty=forecast&param=GeopHeight,Temperature,Pressure,Humidity,WindDirection,WindSpeedMS,WindUMS,WindVMS,MaximumWind,WindGust,DewPoint,TotalCloudCover,WeatherSymbol3,LowCloudCover,MediumCloudCover,HighCloudCover,Precipitation1h,PrecipitationAmount,RadiationGlobalAccumulation,RadiationLWAccumulation,RadiationNetSurfaceLWAccumulation,RadiationNetSurfaceSWAccumulation,RadiationDiffuseAccumulation,LandSeaMask&language=eng


,name,data
2019-04-29 22:00:00+03:00,Temperature,6.6
2019-04-29 23:00:00+03:00,Temperature,5.15
2019-04-30 00:00:00+03:00,Temperature,4.1
2019-04-30 01:00:00+03:00,Temperature,3.57
2019-04-30 02:00:00+03:00,Temperature,3.34
2019-04-30 03:00:00+03:00,Temperature,3.14
2019-04-30 04:00:00+03:00,Temperature,2.83
2019-04-30 05:00:00+03:00,Temperature,2.7
2019-04-30 06:00:00+03:00,Temperature,2.42
2019-04-30 07:00:00+03:00,Temperature,2.81
